In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Data Processing

In [26]:
column_names = ['name_of_surface_type', 'name_of_active_site_type']
for i in range(1,61):
    column_names.append('element_of_atom_'+str(i))
    column_names.append('CN_of_atom_'+str(i))
column_names.append('adsorption_energy')
data = pd.read_csv('journal_data/data.csv', names=column_names, header=None)

In [27]:
data

,name_of_surface_type,name_of_active_site_type,element_of_atom_1,CN_of_atom_1,element_of_atom_2,CN_of_atom_2,element_of_atom_3,CN_of_atom_3,element_of_atom_4,CN_of_atom_4,...,CN_of_atom_56,element_of_atom_57,CN_of_atom_57,element_of_atom_58,CN_of_atom_58,element_of_atom_59,CN_of_atom_59,element_of_atom_60,CN_of_atom_60,adsorption_energy
0,HEA-532,6-8,0.0,1.0,0.0,8.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.521873
1,HEA-532,6-8,0.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.471441
2,HEA-532,6-8,0.0,2.0,3.0,8.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.993456
3,HEA-532,6-8,0.0,2.0,3.0,8.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.905204
4,HEA-532,6-8,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.757654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,RuRh-110,7-7,0.0,1.0,0.0,7.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.376493
1366,RuRh-110,7-7,0.0,0.0,0.0,7.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.590500
1367,RuRh-110,7-7,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.560202
1368,RuRh-110,7-7,0.0,0.0,0.0,7.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.637736


In [58]:
type(data.loc[0, 'element_of_atom_1'])

numpy.float64

In [64]:
data.describe()

,element_of_atom_1,CN_of_atom_1,element_of_atom_2,CN_of_atom_2,element_of_atom_3,CN_of_atom_3,element_of_atom_4,CN_of_atom_4,element_of_atom_5,CN_of_atom_5,...,CN_of_atom_56,element_of_atom_57,CN_of_atom_57,element_of_atom_58,CN_of_atom_58,element_of_atom_59,CN_of_atom_59,element_of_atom_60,CN_of_atom_60,adsorption_energy
count,1370.000000,1370.000000,1370.000000,1370.000000,1370.0,1370.000000,1370.000000,1370.000000,1370.000000,1370.0,...,1370.0,1370.0,1370.0,1370.0,1370.0,1370.0,1370.0,1370.0,1370.0,1370.000000
mean,0.332117,1.056934,1.433212,7.747445,1.0,0.398540,1.164234,1.672993,7.705839,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.998641
std,0.471145,0.750728,1.689273,1.286162,0.0,0.489776,0.725235,1.766088,1.297988,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.422185
min,0.000000,0.000000,0.000000,6.000000,1.0,0.000000,0.000000,0.000000,6.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.052572
25%,0.000000,0.000000,0.000000,7.000000,1.0,0.000000,1.000000,0.000000,7.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.324322
50%,0.000000,1.000000,0.000000,7.000000,1.0,0.000000,1.000000,1.500000,7.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.006455
75%,1.000000,2.000000,3.000000,8.000000,1.0,1.000000,2.000000,3.000000,8.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.675213
max,1.000000,2.000000,4.500000,10.000000,1.0,1.000000,2.000000,4.500000,10.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036113


In [28]:
features = data.iloc[:,2:122]
labels = data.iloc[:,122]

# Convert to numpy arrays
features = np.array(features)
labels = np.array(labels)


In [29]:
from sklearn.model_selection import train_test_split


# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)



Training data shape: (1096, 120)
Testing data shape: (274, 120)


## 2. Model Definition

In [50]:
import torch
import torch.nn as nn

# Define a simple neural network with 3 fully connected layers
class SimpleNeuralNetwork(nn.Module):
    def __init__(self):
        self.name = "SimpleNeuralNetwork"
        super(SimpleNeuralNetwork, self).__init__()

        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=120, out_features=240)  # First layer
        self.fc2 = nn.Linear(in_features=240, out_features=64)  # Second layer
        self.fc3 = nn.Linear(in_features=64, out_features=32)  # Third layer
        self.fc4 = nn.Linear(in_features=32, out_features=1)
        self.relu = nn.ReLU()  # Activation function

    def forward(self, x):
        # Define the forward pass
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [31]:
def get_accuracy(model, train_loader, val_loader, train=False):
    if train:
        data = train_loader
    else:
        data = val_loader
        
    correct = 0
    total = 0
    for imgs, labels in data:
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total


In [32]:
cuda_available = torch.cuda.is_available()

cuda_available

True

In [53]:
def train_model(model, X_train, Y_train, X_test, Y_test, learning_rate=0.001, epochs=10):
    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        # Convert data to torch tensors
        inputs = torch.tensor(X_train, dtype=torch.float32)
        labels = torch.tensor(Y_train, dtype=torch.int64)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print statistics
        if (epoch + 1) % 1 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        inputs = torch.tensor(X_test, dtype=torch.float32)
        labels = torch.tensor(Y_test, dtype=torch.int64)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

In [54]:
model = SimpleNeuralNetwork()

In [55]:
train_model(model, X_train, Y_train, X_test, Y_test)

e:\Documents\MSE465 Project\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1096])) that is different to the input size (torch.Size([1096, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float

In [62]:
type(X_test[0][0])

numpy.float64